In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

url = "https://www.foodsafetykorea.go.kr/portal/healthyfoodlife/searchHomeHF.do?menu_grp=MENU_NEW01&menu_no=2823"
url2 = "https://www.foodsafetykorea.go.kr/portal/healthyfoodlife/searchHomeHFDetail.do"

detailURL = 'https://www.foodsafetykorea.go.kr/portal/healthyfoodlife/searchHomeHFDetail.do'

html = urlopen(url)
bsObject = BeautifulSoup(html, "html.parser")

#print(bsObject)


In [2]:
import requests
import json
import pandas as pd


url = 'https://www.foodsafetykorea.go.kr/portal/healthyfoodlife/searchHomeHFProc.do'

#32419개 가져옴
i=32419

dic={"menu_no":"2823",
     "menu_grp": "MENU_NEW01",
     "menuNm":"",
     "copyUrl":"https://www.foodsafetykorea.go.kr:443/portal/healthyfoodlife/searchHomeHF.do?menu_grp=MENU_NEW01&menu_no=2823",
     "mberld":"",
     "mberNo":"",
     "favorListCnt":"0",
     "search_code":"01",
     "search_word":"",
     "start_idx": "1",
     "show_cnt":str(i)
    }

result = []
requestIter = 1
while True:
    try:
        res = requests.post(url, data=dic)
        break
    except:
        requestIter += 1
        if(requestIter > 100):
            print("request 요청 초과")
            break
        print("request 오류")
        print("request 재요청")
        continue
result += json.loads(res.text)


df2 = pd.DataFrame(result)
prd_no_list = df2['prdlst_report_ledg_no']
#print(df)    
#print(prd_no_list)
#df.to_excel("test1.xlsx")
print("신고번호 가져오기 완료")

신고번호 가져오기 완료


In [ ]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

detailURL = 'https://www.foodsafetykorea.go.kr/portal/healthyfoodlife/searchHomeHFDetail.do'

dic={
    "prdlstReportLedgNo": prd_no_list[0],
     "search_word":"",
     "search_code":"01",
     "start_idx": "",
     "menu_no":"2823",
     "menu_grp": "MENU_NEW01",
    }

result = []
requestIter = 1
while True:
    try:
        res = requests.post(detailURL, data=dic)
        break
    except:
        requestIter += 1
        if(requestIter > 100):
            print("request 요청 초과")
            break
        print("request 오류")
        print("request 재요청")
        continue
html_doc = res.text
soup = BeautifulSoup(html_doc, 'html.parser')
#print(soup.prettify())

cindex = []

for title in soup.find_all('th'):
    cindex.append(title.get_text()) 

cindex = cindex[2:-6]    
#print(cindex)

df = pd.DataFrame(columns=cindex)
#print(df)
for prdNo in tqdm(prd_no_list):
    dic={
    "prdlstReportLedgNo": prdNo,
     "search_word":"",
     "search_code":"01",
     "start_idx": "",
     "menu_no":"2823",
     "menu_grp": "MENU_NEW01",
    }
    result = []
    requestIter = 1
    while True:
        try:
            res = requests.post(detailURL, data=dic)
            break
        except:
            requestIter += 1
            if(requestIter > 100):
                print("request 요청 초과")
                break
            print("request 오류")
            print("request 재요청")
            continue
    html_doc = res.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    datalist = []
    for DATA in soup.find_all('td'):
        #print(DATA.get_text())
        datalist.append(DATA.get_text())
    datalist = datalist[:-3]
    #print(datalist)
    temp = pd.DataFrame(data= [datalist], columns = cindex)
    df = df.append(temp)
#print(df)
df = df.drop('등록일자', axis=1)
df = df.drop('유통기한', axis=1)
df = df.drop('포장재질(방법)', axis=1)
df = df.drop('보존 및 유통기준', axis=1)
df = df.drop('기능성 내용', axis=1)
df = df.drop('기준 및 규격', axis=1)

df['URL'] = url
df['추출 후 남은 데이터'] = "#none" #요건 그냥 추가 ㅎ 
df.rename(columns={'업소명':'제조사','신고번호':'제품코드','성상':'제형',
                   '섭취량/섭취 방법':'섭취방법','섭취시주의사항':'주의 사항'}, inplace =True)
#df = df[['제품코드','제품명','제조사','제형','섭취방법','주의 사항','URL','추출 후 남은 데이터']]

#원재료명 가져오기
rawURL = 'https://www.foodsafetykorea.go.kr/portal/healthyfoodlife/searchHfPrdlstRawmtrl.do'

rawList=[]

for CODE in df['제품코드']:
    dic={"prdlst_report_no": str(CODE)
        }
    result = []
    requestIter = 1
    while True:
        try:
            res = requests.post(rawURL, data=dic)
            break
        except:
            requestIter += 1
            if(requestIter > 100):
                print("request 요청 초과")
                break
            print("request 오류")
            print("request 재요청")
            continue
    result += json.loads(res.text)
    #print(result)
    df3 = pd.DataFrame(result)
    rString=[]
    for raws in df3['rawmtrl_nm']:
        rString.append(raws)
    rString = ",".join(rString)
    rawList.append(rString)

df['원재료명'] = rawList

cutlist=[]
for type in df['제형']:
    cutlist.append( type.split()[-1] )
#print(cutlist)
df['제형'] = cutlist

foodcode=[]
for type in df['제품코드']:
    foodcode.append("kfood-"+str(type))
df['제품코드'] = foodcode


df = df[['제품코드','제품명','제조사','제형','섭취방법','주의 사항','원재료명','URL','추출 후 남은 데이터']]

df.to_excel("result(식품안전나라).xlsx", index = False )

print("완료")

  0%|                                                                            | 39/32419 [00:40<10:07:48,  1.13s/it]